In [1]:
import pandas as pd
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from IPython.display import display, Markdown
from langchain.callbacks import get_openai_callback
import os

In [2]:
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_SESSION"] = "csv_query"
os.environ["OPENAI_API_KEY"] = "open api key"

In [3]:
df = pd.read_csv("data/BTC.csv")
df.head()

,ticker,date,open,high,low,close
0,BTC,2022-07-01,18797.2,20870.1,18719.9,19375.7
1,BTC,2022-07-02,19375.7,19438.2,18980.4,19332.0
2,BTC,2022-07-03,19332.0,19614.2,18773.6,19279.7
3,BTC,2022-07-04,19275.8,20095.2,19030.4,19934.8
4,BTC,2022-07-05,19934.8,20723.9,19283.9,20350.7


In [4]:
df.shape

(370, 6)

In [5]:
davinci_agent = create_csv_agent(
    OpenAI(temperature=0),
    "data/BTC.csv",
    verbose=True
)
gpt_3_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    "data/BTC.csv",
    verbose=True
)

In [6]:
gpt_3_agent

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df_head': '|    | ticker   | date       |    open |    high |     low |   close |\n|---:|:---------|:-----------|--------:|--------:|--------:|--------:|\n|  0 | BTC      | 2022-07-01 | 18797.2 | 20870.1 | 18719.9 | 19375.7 |\n|  1 | BTC      | 2022-07-02 | 19375.7 | 19438.2 | 18980.4 | 19332   |\n|  2 | BTC      | 2022-07-03 | 19332   | 19614.2 | 18773.6 | 19279.7 |\n|  3 | BTC      | 2022-07-04 | 19275.8 | 20095.2 | 19030.4 | 19934.8 |\n|  4 | BTC      | 2022-07-05 | 19934.8 | 20723.9 | 19283.9 | 20350.7 |'}, template='\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:

In [7]:
gpt_3_agent.agent.llm_chain.prompt

PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df_head': '|    | ticker   | date       |    open |    high |     low |   close |\n|---:|:---------|:-----------|--------:|--------:|--------:|--------:|\n|  0 | BTC      | 2022-07-01 | 18797.2 | 20870.1 | 18719.9 | 19375.7 |\n|  1 | BTC      | 2022-07-02 | 19375.7 | 19438.2 | 18980.4 | 19332   |\n|  2 | BTC      | 2022-07-03 | 19332   | 19614.2 | 18773.6 | 19279.7 |\n|  3 | BTC      | 2022-07-04 | 19275.8 | 20095.2 | 19030.4 | 19934.8 |\n|  4 | BTC      | 2022-07-05 | 19934.8 | 20723.9 | 19283.9 | 20350.7 |'}, template='\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before

In [8]:
display(Markdown(gpt_3_agent.agent.llm_chain.prompt.template))


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}

In [20]:
with get_openai_callback() as cb:
    gpt_3_agent.run("How many rows and columns the data has")
    print(cb)



> Entering new  chain...
Thought: To determine the number of rows and columns in the dataframe, I can use the `shape` attribute of the dataframe.

Action: python_repl_ast
Action Input: df.shape
Observation: (370, 6)
Thought:The dataframe has 370 rows and 6 columns.
Final Answer: The dataframe has 370 rows and 6 columns.

> Finished chain.
Tokens Used: 1019
	Prompt Tokens: 956
	Completion Tokens: 63
Successful Requests: 2
Total Cost (USD): $0.0015600000000000002


In [21]:
with get_openai_callback() as cb:
    davinci_agent.run("How many rows and columns the data has")
    print(cb)



> Entering new  chain...
Thought: I need to find out the shape of the dataframe
Action: python_repl_ast
Action Input: df.shape
Observation: (370, 6)
Thought: I now know the final answer
Final Answer: The data has 370 rows and 6 columns.

> Finished chain.
Tokens Used: 976
	Prompt Tokens: 928
	Completion Tokens: 48
Successful Requests: 2
Total Cost (USD): $0.019520000000000003


In [11]:
with get_openai_callback() as cb:
    gpt_3_agent.run("What is the average of high price during february 2023")
    print(cb)



> Entering new  chain...
Thought: I need to filter the dataframe to only include the rows from February 2023, and then calculate the average of the "high" column.

Action: Use pandas' `loc` function to filter the dataframe and calculate the average.

Action Input: `df.loc[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-02-28'), 'high'].mean()`

Observation: Use pandas' `loc` function to filter the dataframe and calculate the average. is not a valid tool, try another one.
Thought:I need to use the `python_repl_ast` tool to execute the pandas command.
Action: Use the `python_repl_ast` tool.
Action Input: `df.loc[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-02-28'), 'high'].mean()`
Observation: Use the `python_repl_ast` tool. is not a valid tool, try another one.
Thought:I need to use the `python_repl` tool to execute the pandas command.
Action: Use the `python_repl` tool.
Action Input: `df.loc[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-02-28'), 'high'].mean()`
Obser

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-FGRdOoshDo4NEIcBpi6pcqZR on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-FGRdOoshDo4NEIcBpi6pcqZR on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

In [9]:
df.loc[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-02-28'), 'high'].mean()-

23730.09285714286

In [10]:
with get_openai_callback() as cb:
    davinci_agent.run("What is the average of high price during february 2023")
    print(cb)



> Entering new  chain...
Thought: I need to filter the dataframe to only include february 2023 and then calculate the average of the high column
Action: python_repl_ast
Action Input: df[df['date'].str.contains('2023-02')]['high'].mean()
Observation: 23730.09285714286
Thought: I now know the final answer
Final Answer: The average of high price during february 2023 is 23730.09285714286.

> Finished chain.
Tokens Used: 1069
	Prompt Tokens: 975
	Completion Tokens: 94
Successful Requests: 2
Total Cost (USD): $0.021380000000000003


In [16]:
with get_openai_callback() as cb:
    davinci_agent.run("What is the difference of price during February and March 2023")
    print(cb)



> Entering new  chain...
Thought: I need to find the difference between the open and close prices for February and March 2023
Action: python_repl_ast
Action Input: df[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-03-31')][['open', 'close']].diff().max()
Observation: open     2642.3
close    2706.3
dtype: float64
Thought: I now know the final answer
Final Answer: The maximum difference between the open and close prices during February and March 2023 is 2706.3.

> Finished chain.
Tokens Used: 1104
	Prompt Tokens: 997
	Completion Tokens: 107
Successful Requests: 2
Total Cost (USD): $0.022080000000000002


In [13]:
with get_openai_callback() as cb:
    gpt_3_agent.run("What is the difference of high price during February and March 2023")
    print(cb)



> Entering new  chain...
Thought: To find the difference in high prices during February and March 2023, I need to filter the dataframe for the relevant dates and then calculate the difference between the maximum high price in February and the maximum high price in March.

Action: Filter the dataframe for February and March 2023.

Action Input: 
```python_repl_ast
df['date'] = pd.to_datetime(df['date']) # Convert date column to datetime
df_filtered = df[(df['date'].dt.year == 2023) & ((df['date'].dt.month == 2) | (df['date'].dt.month == 3))]
```

Observation: Filter the dataframe for February and March 2023. is not a valid tool, try another one.
Thought:I need to use the `query` method to filter the dataframe for February and March 2023.

Action: Filter the dataframe for February and March 2023 using the `query` method.

Action Input: 
```python_repl_ast
df['date'] = pd.to_datetime(df['date']) # Convert date column to datetime
df_filtered = df.query('date.dt.year == 2023 and (date.dt.

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-FGRdOoshDo4NEIcBpi6pcqZR on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-FGRdOoshDo4NEIcBpi6pcqZR on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

In [15]:
df.loc[(df['date'] >= '2023-02-01') & (df['date'] <= '2023-02-28'), 'high'].mean()-df.loc[(df['date'] >= '2023-03-01') & (df['date'] <= '2023-03-31'), 'high'].mean()

-1949.1716589861753

In [17]:
with get_openai_callback() as cb:
    gpt_3_agent.run("What is % increase of closing price from 2022 to 2023")
    print(cb)



> Entering new  chain...
Thought: To calculate the percentage increase of the closing price from 2022 to 2023, I need to find the closing prices for both years and then calculate the percentage increase.

Action: I will filter the dataframe to include only the rows for the year 2022 and 2023.

Action Input: 
```python_repl_ast
df_2022 = df[df['date'].dt.year == 2022]
df_2023 = df[df['date'].dt.year == 2023]
```

Observation: I will filter the dataframe to include only the rows for the year 2022 and 2023. is not a valid tool, try another one.
Thought:I need to use the `pandas` library to filter the dataframe based on the year.
Action: I will use the `pandas` library to filter the dataframe based on the year.
Action Input: 
```python_repl_ast
df_2022 = df[df['date'].dt.year == 2022]
df_2023 = df[df['date'].dt.year == 2023]
```
Observation: I will use the `pandas` library to filter the dataframe based on the year. is not a valid tool, try another one.
Thought:I need to use the `pandas` 


KeyboardInterrupt



In [18]:
with get_openai_callback() as cb:
    davinci_agent.run("What is % increase of closing price from 2022 to 2023")
    print(cb)



> Entering new  chain...
Thought: I need to find the closing price for 2022 and 2023
Action: python_repl_ast
Action Input: df[df['date'] == '2022-07-01']['close']
Observation: 0    19375.7
Name: close, dtype: float64
Thought: I need to find the closing price for 2023
Action: python_repl_ast
Action Input: df[df['date'] == '2023-07-01']['close']
Observation: 365    30584.7
Name: close, dtype: float64
Thought: I now know the final answer
Final Answer: 57.7%

> Finished chain.
Tokens Used: 1647
	Prompt Tokens: 1546
	Completion Tokens: 101
Successful Requests: 3
Total Cost (USD): $0.03294
